In [2]:
import pandas as pd
import numpy as np
import re

In [50]:
traintanic = pd.read_csv('train.csv')
testanic = pd.read_csv('test.csv')

Initial features:
Ticket class, sex, age, parent-child, sibling-spouse, nationality (split off last names and classify non-Anglos by name ending)

Start with decision tree, then do KNN, then SVM
See what happens when combined

To do: separate out deck from cabin number

Check on how exactly the ship sank - 

Create column for young children (try different limit downwards from 16)

Ignore ticket & fare

Ignore port of embarkation


'Foreign' name endings:
off
ov
cher
sky
ski
ig
nen
sen
ik
strom
ic
sen
blom
lund
og

In [52]:
traintanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [53]:
traintanic['Last_Name'] = traintanic['Name'].str.split(',', n = 1)

In [54]:
traintanic['Last_Name'] = traintanic['Name'].str.split(',', n = 1, expand = True)

In [55]:
traintanic['Is Non-Anglo Western?'] = traintanic.Name.apply\
(lambda x: 1 if re.compile('adt,|berg,|berger,|blom,|burg,|burger,|a,|o,|elle,|elli,|ehr,|ig,|ila,|cher,\
|nen,|sen,|ik,|ek,|og,|our,|gen,|lund,|strom,|sson,|sen,|vedt,|stad,|stedt,| van |qvist,|quist,\
            |aki,|feld,|mann,|gli,|glin,|ahl,|ouf,| Van| Vander').search(x) else 0)

testanic['Is Non-Anglo Western?'] = testanic.Name.apply\
(lambda x: 1 if re.compile('adt,|berg,|berger,|blom,|burg,|burger,|a,|o,|elle,|elli,|ehr,|ig,|ila,|cher,\
|nen,|sen,|ik,|ek,|og,|our,|gen,|lund,|strom,|sson,|sen,|vedt,|stad,|stedt,| van |qvist,|quist,\
            |aki,|feld,|mann,|gli,|glin,|ahl,|ouf,| Van| Vander').search(x) else 0)

In [56]:
traintanic['Is Eastern?'] = traintanic.Name.apply\
(lambda x: 1 if re.compile('ac,|off,|eff,|if,|ov,|ic,|sky,|ski,|u,|ib,|os,|oub,').search(x) else 0)

testanic['Is Eastern?'] = testanic.Name.apply\
(lambda x: 1 if re.compile('ac,|off,|eff,|if,|ov,|ic,|sky,|ski,|u,|ib,|os,|oub,').search(x) else 0)

In [57]:
traintanic['Is 1st Class?'] = traintanic.Pclass.apply(lambda x: 1 if x == 1 else 0)

testanic['Is 1st Class?'] = testanic.Pclass.apply(lambda x: 1 if x == 1 else 0)

In [58]:
traintanic['Is 3rd Class?'] = traintanic.Pclass.apply(lambda x: 1 if x == 3 else 0)

testanic['Is 3rd Class?'] = testanic.Pclass.apply(lambda x: 1 if x == 3 else 0)

In [59]:
traintanic['Deck'] = traintanic.Cabin.str.get(0)
traintanic['Cabin'] = traintanic.Cabin.astype('object')

testanic['Deck'] = testanic.Cabin.str.get(0)
testanic['Cabin'] = testanic.Cabin.astype('object')

In [60]:
traintanic['Cabin_No'] = traintanic['Cabin'].str[1:4]

testanic['Cabin_No'] = testanic['Cabin'].str[1:4]

In [61]:
traintanic.Deck = traintanic.Deck.fillna(0)
traintanic.Cabin_No = traintanic.Cabin_No.fillna(0)

testanic.Deck = testanic.Deck.fillna(0)
testanic.Cabin_No = testanic.Cabin_No.fillna(0)

In [62]:
traintanic['Is Child?'] = traintanic.Age.apply(lambda x: 1 if x <= 12 else 0)
traintanic['Is Young Adult?'] = traintanic.Age.apply(lambda x: 1 if x in range(13,31) else 0)

testanic['Is Child?'] = testanic.Age.apply(lambda x: 1 if x <= 12 else 0)
testanic['Is Young Adult?'] = testanic.Age.apply(lambda x: 1 if x in range(13,31) else 0)

In [63]:
traintanic['Is Not Young?'] = traintanic.Age.apply(lambda x: 1 if x > 40 else 0)

testanic['Is Not Young?'] = testanic.Age.apply(lambda x: 1 if x > 40 else 0)

In [64]:
traintanic['Big Family?'] = traintanic.Parch.apply(lambda x: 1 if x > 2 else 0)

testanic['Big Family?'] = testanic.Parch.apply(lambda x: 1 if x > 2 else 0)

In [65]:
traintanic[['Is Child?','Is Young Adult?']].fillna(0, inplace=True)

testanic[['Is Child?','Is Young Adult?']].fillna(0, inplace=True)

C:\Users\teren\miniconda3\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [66]:
traintanic.Cabin_No.replace(to_replace=' G7', value=7, inplace=True)
traintanic.Cabin_No.replace(to_replace=' G6', value=6, inplace=True)
traintanic.Cabin_No.replace(to_replace=' E6', value=6, inplace=True)
traintanic.Cabin_No.replace(to_replace='', value=0, inplace=True)

In [67]:
testanic.Cabin_No.replace(to_replace=' [A-Z]', value='', inplace=True, regex=True)
testanic.Cabin_No.replace(to_replace='', value='0', inplace=True, regex=True)

In [68]:
traintanic['Cabin_No'] = traintanic['Cabin_No'].astype('int64')

testanic['Cabin_No'] = testanic['Cabin_No'].astype('int64')

In [69]:
traintanic['Cabin Fore?'] = traintanic['Cabin_No'].apply(lambda x: 1 if x in range(66) else 0)

testanic['Cabin Fore?'] = testanic['Cabin_No'].apply(lambda x: 1 if x in range(66) else 0)

In [70]:
traintanic['Cabin Aft?'] = traintanic['Cabin_No'].apply(lambda x: 1 if x > 80 else 0)

testanic['Cabin Aft?'] = testanic['Cabin_No'].apply(lambda x: 1 if x > 80 else 0)

In [71]:
traintanic['Cabin Amidships?'] = traintanic['Cabin_No'].apply(lambda x: 1 if x in range(66,81) else 0)

testanic['Cabin Amidships?'] = testanic['Cabin_No'].apply(lambda x: 1 if x in range(66,81) else 0)

In [72]:
traintanic['Sex-int'] = traintanic['Sex'].apply(lambda x: 1 if x == 'female' else 0)

testanic['Sex-int'] = testanic['Sex'].apply(lambda x: 1 if x == 'female' else 0)

In [73]:
# Anyone who has seen the film might suspect women and children had a higher than average
# chance of surviving the sinking. Though what if some of our entries here are missing a value for 'Age'?
# We might in those cases omit to take any age significance into account for those datapoints.

# Below I have filtered the training set for passengers without a recorded age whose identity as 
# young woman or boy we can nevertheless deduce through the titles 'Miss' and 'Master'.

# This filter still cannot address young men without a recorded age as the title 'Mr.' can apply to a male
# of any age past boyhood. I will consider other possibilities to identify such passengers.

traintanic[(traintanic['Age'].isna()) & traintanic['Name'].str.contains('Master|Miss|Mrs.')]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Deck,Cabin_No,Is Child?,Is Young Adult?,Is Not Young?,Big Family?,Cabin Fore?,Cabin Aft?,Cabin Amidships?,Sex-int
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,...,0,0,0,0,0,0,1,0,0,1
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,...,0,0,0,0,0,0,1,0,0,1
31,32,1,1,"Spencer, Mrs. William Augustus (Marie Eugenie)",female,NaN,1,0,PC 17569,146.5208,...,B,78,0,0,0,0,0,0,1,1
32,33,1,3,"Glynn, Miss. Mary Agatha",female,NaN,0,0,335677,7.7500,...,0,0,0,0,0,0,1,0,0,1
47,48,1,3,"O'Driscoll, Miss. Bridget",female,NaN,0,0,14311,7.7500,...,0,0,0,0,0,0,1,0,0,1
65,66,1,3,"Moubarek, Master. Gerios",male,NaN,1,1,2661,15.2458,...,0,0,0,0,0,0,1,0,0,0
82,83,1,3,"McDermott, Miss. Brigdet Delia",female,NaN,0,0,330932,7.7875,...,0,0,0,0,0,0,1,0,0,1
109,110,1,3,"Moran, Miss. Bertha",female,NaN,1,0,371110,24.1500,...,0,0,0,0,0,0,1,0,0,1
128,129,1,3,"Peter, Miss. Anna",female,NaN,1,1,2668,22.3583,...,F,6,0,0,0,0,1,0,0,1
140,141,0,3,"Boulos, Mrs. Joseph (Sultana)",female,NaN,0,2,2678,15.2458,...,0,0,0,0,0,0,1,0,0,1


In [74]:
traintanic['Age'] = \
traintanic.apply(lambda row: traintanic[traintanic.Name.str.contains('Master')]['Age'].median() if \
                 re.compile('Master').search(row['Name']) and np.isnan(row['Age']) \
                 else row['Age'], axis=1)

testanic['Age'] = \
testanic.apply(lambda row: testanic[testanic.Name.str.contains('Master')]['Age'].median() if \
                 re.compile('Master').search(row['Name']) and np.isnan(row['Age']) \
                 else row['Age'], axis=1)

In [75]:
traintanic['Age'] = traintanic.apply(lambda row: traintanic.Age.median() if \
                row['Sex-int']==1 and np.isnan(row['Age']) else row['Age'], axis=1)

testanic['Age'] = testanic.apply(lambda row: testanic.Age.median() if \
                row['Sex-int']==1 and np.isnan(row['Age']) else row['Age'], axis=1)

In [76]:
traintanic['Age'] = \
traintanic.apply(lambda row: traintanic.Age.median() if row['Sex-int']==0 and \
                 re.compile('Mr\.|Rev\.|Dr\.').search(row['Name']) and np.isnan(row['Age']) \
                 else row['Age'], axis=1)

testanic['Age'] = \
testanic.apply(lambda row: testanic.Age.median() if row['Sex-int']==0 and \
                 re.compile('Mr\.|Rev\.|Dr\.').search(row['Name']) and np.isnan(row['Age']) \
                 else row['Age'], axis=1)

In [77]:
traintanic[traintanic['Age'].isna()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Deck,Cabin_No,Is Child?,Is Young Adult?,Is Not Young?,Big Family?,Cabin Fore?,Cabin Aft?,Cabin Amidships?,Sex-int


In [78]:
traintanic[traintanic.Name.str.contains('Master')]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Deck,Cabin_No,Is Child?,Is Young Adult?,Is Not Young?,Big Family?,Cabin Fore?,Cabin Aft?,Cabin Amidships?,Sex-int
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.00,3,1,349909,21.0750,...,0,0,1,0,0,0,1,0,0,0
16,17,0,3,"Rice, Master. Eugene",male,2.00,4,1,382652,29.1250,...,0,0,1,0,0,0,1,0,0,0
50,51,0,3,"Panula, Master. Juha Niilo",male,7.00,4,1,3101295,39.6875,...,0,0,1,0,0,0,1,0,0,0
59,60,0,3,"Goodwin, Master. William Frederick",male,11.00,5,2,CA 2144,46.9000,...,0,0,1,0,0,0,1,0,0,0
63,64,0,3,"Skoog, Master. Harald",male,4.00,3,2,347088,27.9000,...,0,0,1,0,0,0,1,0,0,0
65,66,1,3,"Moubarek, Master. Gerios",male,3.50,1,1,2661,15.2458,...,0,0,0,0,0,0,1,0,0,0
78,79,1,2,"Caldwell, Master. Alden Gates",male,0.83,0,2,248738,29.0000,...,0,0,1,0,0,0,1,0,0,0
125,126,1,3,"Nicola-Yarred, Master. Elias",male,12.00,1,0,2651,11.2417,...,0,0,1,0,0,0,1,0,0,0
159,160,0,3,"Sage, Master. Thomas Henry",male,3.50,8,2,CA. 2343,69.5500,...,0,0,0,0,0,0,1,0,0,0
164,165,0,3,"Panula, Master. Eino Viljami",male,1.00,4,1,3101295,39.6875,...,0,0,1,0,0,0,1,0,0,0


In [81]:
traintanic['Embarked'].fillna('S', inplace=True)

In [82]:
traintanic.Embarked

0      S
1      C
2      S
3      S
4      S
      ..
886    S
887    S
888    S
889    C
890    Q
Name: Embarked, Length: 891, dtype: object

In [93]:
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
Embarked_array = np.array(traintanic['Embarked'])
Embarked_1hot_matrix = cat_encoder.fit_transform(Embarked_array.reshape(-1,1))

In [97]:
Embarked_1hot_array = Embarked_1hot_matrix.toarray()

In [99]:
Embarked_1hot_array.shape

(891, 3)

In [95]:
cat_encoder.categories_

[array(['C', 'Q', 'S'], dtype=object)]

In [100]:
traintanic['Embarked C'] = Embarked_1hot_array[:, 0]
traintanic['Embarked Q'] = Embarked_1hot_array[:, 1]
traintanic['Embarked S'] = Embarked_1hot_array[:, 2]

In [102]:
traintanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PassengerId            891 non-null    int64  
 1   Survived               891 non-null    int64  
 2   Pclass                 891 non-null    int64  
 3   Name                   891 non-null    object 
 4   Sex                    891 non-null    object 
 5   Age                    891 non-null    float64
 6   SibSp                  891 non-null    int64  
 7   Parch                  891 non-null    int64  
 8   Ticket                 891 non-null    object 
 9   Fare                   891 non-null    float64
 10  Cabin                  204 non-null    object 
 11  Embarked               891 non-null    object 
 12  Last_Name              891 non-null    object 
 13  Is Non-Anglo Western?  891 non-null    int64  
 14  Is Eastern?            891 non-null    int64  
 15  Is 1st

In [89]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [92]:
traintanic.Embarked_1hot

0        (0, 2)\t1.0\n  (1, 0)\t1.0\n  (2, 2)\t1.0\n ...
1        (0, 2)\t1.0\n  (1, 0)\t1.0\n  (2, 2)\t1.0\n ...
2        (0, 2)\t1.0\n  (1, 0)\t1.0\n  (2, 2)\t1.0\n ...
3        (0, 2)\t1.0\n  (1, 0)\t1.0\n  (2, 2)\t1.0\n ...
4        (0, 2)\t1.0\n  (1, 0)\t1.0\n  (2, 2)\t1.0\n ...
                             ...                        
886      (0, 2)\t1.0\n  (1, 0)\t1.0\n  (2, 2)\t1.0\n ...
887      (0, 2)\t1.0\n  (1, 0)\t1.0\n  (2, 2)\t1.0\n ...
888      (0, 2)\t1.0\n  (1, 0)\t1.0\n  (2, 2)\t1.0\n ...
889      (0, 2)\t1.0\n  (1, 0)\t1.0\n  (2, 2)\t1.0\n ...
890      (0, 2)\t1.0\n  (1, 0)\t1.0\n  (2, 2)\t1.0\n ...
Name: Embarked_1hot, Length: 891, dtype: object

In [104]:
forest = RandomForestClassifier(\
        n_estimators = 2000, random_state = 1, \
            max_samples = 0.8, min_samples_leaf = 5)
traintanic_data, valitanic_data, traintanic_labels, valitanic_labels = train_test_split(traintanic[[\
'Pclass','Sex-int','SibSp','Parch','Embarked C','Embarked Q', 'Embarked S',
    'Is Non-Anglo Western?','Is Eastern?',\
        'Age']],\
           traintanic['Survived'], test_size = 0.1)
forest.fit(traintanic_data, traintanic_labels)

print(forest.score(valitanic_data, valitanic_labels))

0.8777777777777778


In [112]:
for i in range(10):
 forest = RandomForestClassifier(\
        n_estimators = 2000, random_state = 1, \
            max_samples = 0.8, min_samples_leaf = 5)
 traintanic_data, valitanic_data, traintanic_labels, valitanic_labels = train_test_split(traintanic[[\
'Pclass','Sex-int','SibSp','Parch','Is Non-Anglo Western?', 'Is Eastern?',
        'Embarked C','Embarked Q', 'Embarked S','Age']],\
           traintanic['Survived'], test_size = 0.1)
 forest.fit(traintanic_data, traintanic_labels)

 print(forest.score(valitanic_data, valitanic_labels))

0.7888888888888889
0.8777777777777778
0.8555555555555555
0.8555555555555555
0.8222222222222222
0.8111111111111111
0.8444444444444444
0.8333333333333334
0.8444444444444444
0.8555555555555555


Removed binary women and children features to incorporate discrete ages; missing-age passengers with title 'Master' assigned the median age of Masters;

All other passengers with missing ages assigned the dataset median

In [57]:
testanic_data = testanic[[\
'Pclass','Sex-int','SibSp','Parch',\
    'Is Non-Anglo Western?','Is Eastern?',\
        'Age']]

In [59]:
testanic_predict = forest.predict(testanic_data)

In [60]:
testanic['Survived'] = testanic_predict